In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

In [8]:
!wget https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip

--2025-10-19 16:54:47--  https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/267273319/c52902a0-e13c-4ca7-92e0-086751098a05?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-19T09%3A49%3A29Z&rscd=attachment%3B+filename%3Dmilvus_docs_2.4.x_en.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-19T08%3A48%3A33Z&ske=2025-10-19T09%3A49%3A29Z&sks=b&skv=2018-11-09&sig=nQ8ssrg5g7wQ9UXCg1TIGmT%2Bd5KilOdrWKO0sWsYy6g%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MDg2NDM4OCwibmJmIjoxNzYwODY0MDg4LCJwYXRoIjoi

In [31]:
!unzip -q milvus_docs_2.4.x_en.zip -d milvus_docs

In [32]:
from glob import glob

text_lines = []

for file_path in glob("milvus_docs/en/faq/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [33]:
len(text_lines)

72

In [34]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [35]:
# from pymilvus import model as milvus_model

# embedding_model = milvus_model.DefaultEmbeddingFunction()

from pymilvus import model as milvus_model

# OpenAI国内代理 https://api.apiyi.com/token
embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
    model_name='text-embedding-3-large', # Specify the model name
    api_key='sk-xsV0B5iUhLxrQJMT0a64Be7939D940Eb8f71A566E4B0052a', # Provide your OpenAI API key
    base_url='https://api.apiyi.com/v1',
    dimensions=512
)

In [36]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

512
[-0.02814663  0.00428726 -0.01852599  0.08190062 -0.03156214 -0.05275258
 -0.04885425  0.12481797 -0.0208328   0.03966279]


In [37]:
test_embedding_0 = embedding_model.encode_queries(["That is a new test"])[0]
print(test_embedding_0[:10])

[-0.04362872  0.01043117 -0.02711612  0.08115138 -0.01261425 -0.05623476
 -0.00316177  0.0844342  -0.01281942  0.05334587]


In [38]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collection"

In [39]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [40]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

In [41]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 72/72 [00:00<00:00, 277054.94it/s]


{'insert_count': 72, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], 'cost': 0}

In [42]:
question = "How is data stored in milvus?"

In [43]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [44]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        " Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###",
        0.7751178741455078
    ],
    [
        "How does Milvus handle vector data types and precision?\n\nMilvus supports Binary, Float

In [45]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [46]:
context

" Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###\nHow does Milvus handle vector data types and precision?\n\nMilvus supports Binary, Float32, Float16, and BFloat16 vector types.\n\n- Binary vectors: Store

In [47]:
question

'How is data stored in milvus?'

In [48]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [49]:
USER_PROMPT

"\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###\nHow does Milvus handle vector data typ

In [50]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

Milvus stores data in two main categories: inserted data and metadata.

- **Inserted data** (including vector data, scalar data, and collection-specific schema) is stored in persistent storage as incremental logs. It supports multiple object storage backends such as MinIO, AWS S3, Google Cloud Storage, Azure Blob Storage, Alibaba Cloud OSS, and Tencent Cloud Object Storage.
- **Metadata** is generated within Milvus and stored in etcd, with each module having its own metadata.

<translated>
Milvus 如何存储数据？

Milvus 存储两种主要类型的数据：插入数据和元数据。

- **插入数据**（包括向量数据、标量数据和集合特定模式）以增量日志的形式存储在持久化存储中。它支持多种对象存储后端，如 MinIO、AWS S3、Google Cloud Storage、Azure Blob Storage、阿里云 OSS 和腾讯云对象存储。
- **元数据** 在 Milvus 内部生成，并存储在 etcd 中，每个模块都有其自己的元数据。
</translated>


In [51]:
from glob import glob

mfd_text_lines = []

for file_path in glob("mfd.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    mfd_text_lines += file_text.split("# ")

In [52]:
len(mfd_text_lines)

30

In [53]:
from pymilvus import MilvusClient

mfd_milvus_client = MilvusClient(uri="./mfd_milvus_demo.db")

mfd_collection_name = "my_mfd_rag_collection"

In [55]:
if mfd_milvus_client.has_collection(mfd_collection_name):
    mfd_milvus_client.drop_collection(mfd_collection_name)

In [56]:
mfd_milvus_client.create_collection(
    collection_name=mfd_collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

In [57]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(mfd_text_lines)

for i, line in enumerate(tqdm(mfd_text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

mfd_milvus_client.insert(collection_name=mfd_collection_name, data=data)

Creating embeddings: 100%|██████████| 30/30 [00:00<00:00, 258907.65it/s]


{'insert_count': 30, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'cost': 0}

In [58]:
question = "民法典的主要内容有哪些？"

In [59]:
search_res = mfd_milvus_client.search(
    collection_name=mfd_collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [60]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u4e2d\u534e\u4eba\u6c11\u5171\u548c\u56fd\u6c11\u6cd5\u5178\n\n##",
        0.5136171579360962
    ],
    [
        "\u7b2c\u4e00\u7ae0 \u4e00\u822c\u89c4\u5b9a\n\n**\u7b2c\u4e8c\u767e\u96f6\u56db\u6761** \u4e3a\u4e86\u660e\u786e\u7269\u7684\u5f52\u5c5e\uff0c\u5145\u5206\u53d1\u6325\u7269\u7684\u6548\u7528\uff0c\u4fdd\u62a4\u6743\u5229\u4eba\u7684\u5408\u6cd5\u6743\u76ca\uff0c\u7ef4\u62a4\u793e\u4f1a\u7ecf\u6d4e\u79e9\u5e8f\uff0c\u5236\u5b9a\u672c\u7f16\u3002\n\n**\u7b2c\u4e8c\u767e\u96f6\u4e94\u6761** \u672c\u7f16\u8c03\u6574\u56e0\u7269\u7684\u5f52\u5c5e\u548c\u5229\u7528\u4ea7\u751f\u7684\u6c11\u4e8b\u5173\u7cfb\u3002\n\n**\u7b2c\u4e8c\u767e\u96f6\u516d\u6761** \u56fd\u5bb6\u575a\u6301\u548c\u5b8c\u5584\u793e\u4f1a\u4e3b\u4e49\u516c\u6709\u5236\u4e3a\u4e3b\u4f53\u3001\u591a\u79cd\u6240\u6709\u5236\u7ecf\u6d4e\u5171\u540c\u53d1\u5c55\u7684\u57fa\u672c\u7ecf\u6d4e\u5236\u5ea6\u3002\n\u56fd\u5bb6\u5de9\u56fa\u548c\u53d1\u5c55\u516c\u6709\u5236\u7ecf\u6d4e\uff0c\u9f13\

In [61]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [62]:
context

'中华人民共和国民法典\n\n##\n第一章 一般规定\n\n**第二百零四条** 为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。\n\n**第二百零五条** 本编调整因物的归属和利用产生的民事关系。\n\n**第二百零六条** 国家坚持和完善社会主义公有制为主体、多种所有制经济共同发展的基本经济制度。\n国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。\n国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。\n\n**第二百零七条** 国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。\n\n**第二百零八条** 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。\n\n**第二百零九条** 不动产物权的设立、变更、转让和消灭，经依法登记，发生效力；未经登记，不发生效力，但是法律另有规定的除外。\n依法属于国家所有的自然资源，所有权可以不登记。\n\n**第二百一十条** 不动产登记，由不动产所在地的登记机构办理。\n国家对不动产实行统一登记制度。统一登记的范围、登记机构和登记办法，由法律、行政法规规定。\n\n**第二百一十一条** 当事人申请登记，应当根据不同登记事项提供材料。\n申请登记材料以及登记事项相关信息，可以公开查询。\n\n**第二百一十二条** 登记机构应当履行下列职责：\n（一）审查申请人提供的材料；\n（二）询问申请人；\n（三）如实、及时登记；\n（四）法律、行政法规规定的其他职责。\n申请登记的不动产存在尚未解决的权属争议的，登记机构应当不予登记，并书面告知申请人。\n\n**第二百一十三条** 登记机构不得有下列行为：\n（一）要求对不动产进行评估；\n（二）以不动产登记为条件收取其他费用；\n（三）超出登记职责范围的其他行为。\n\n**第二百一十四条** 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。\n\n**第二百一十五条** 不动产登记簿由登记机构管理。\n不动产登记簿应当采用纸质形式或者电子形式。\n不动产登记簿采用电子形式的，应当备份。\n\n**第二百一十六条** 不动产登记簿是物权归属和内容的根据。\n不动产登记簿记载的事

In [63]:
question

'民法典的主要内容有哪些？'

In [64]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [65]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n中华人民共和国民法典\n\n##\n第一章 一般规定\n\n**第二百零四条** 为了明确物的归属，充分发挥物的效用，保护权利人的合法权益，维护社会经济秩序，制定本编。\n\n**第二百零五条** 本编调整因物的归属和利用产生的民事关系。\n\n**第二百零六条** 国家坚持和完善社会主义公有制为主体、多种所有制经济共同发展的基本经济制度。\n国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。\n国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。\n\n**第二百零七条** 国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。\n\n**第二百零八条** 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。\n\n**第二百零九条** 不动产物权的设立、变更、转让和消灭，经依法登记，发生效力；未经登记，不发生效力，但是法律另有规定的除外。\n依法属于国家所有的自然资源，所有权可以不登记。\n\n**第二百一十条** 不动产登记，由不动产所在地的登记机构办理。\n国家对不动产实行统一登记制度。统一登记的范围、登记机构和登记办法，由法律、行政法规规定。\n\n**第二百一十一条** 当事人申请登记，应当根据不同登记事项提供材料。\n申请登记材料以及登记事项相关信息，可以公开查询。\n\n**第二百一十二条** 登记机构应当履行下列职责：\n（一）审查申请人提供的材料；\n（二）询问申请人；\n（三）如实、及时登记；\n（四）法律、行政法规规定的其他职责。\n申请登记的不动产存在尚未解决的权属争议的，登记机构应当不予登记，并书面告知申请人。\n\n**第二百一十三条** 登记机构不得有下列行为：\n（一）要求对不动产进行评估；\n（二）以不动产登记为条件收取其他费用；\n（三）超出登记职责范围的其他行为。\n\n**第二百一十四条** 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。\n\n

In [66]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，中华人民共和国民法典的主要内容涉及物权和合同两个重要领域。具体包括：

1. **物权编**：调整因物的归属和利用产生的民事关系，规定了不动产和动产的登记、设立、变更、转让和消灭等规则。例如，不动产权利的设立需依法登记，动产物权的设立自交付时生效。此外，还包括善意取得、遗失物处理、征收补偿等内容。

2. **合同编**：调整因合同产生的民事关系，规定了合同的订立、内容、履行、违约责任等。例如，合同可以采用书面、口头或其他形式订立，内容包括标的、数量、质量等条款。还涉及要约、承诺、定金、违约金等具体规则。

这些规定旨在明确物的归属、保护权利人的合法权益、维护社会经济秩序，并促进社会主义现代化建设。

<translated>
中华人民共和国民法典的主要内容涉及物权和合同两个重要领域。具体包括：

1. **物权编**：调整因物的归属和利用产生的民事关系，规定了不动产和动产的登记、设立、变更、转让和消灭等规则。例如，不动产权利的设立需依法登记，动产物权的设立自交付时生效。此外，还包括善意取得、遗失物处理、征收补偿等内容。

2. **合同编**：调整因合同产生的民事关系，规定了合同的订立、内容、履行、违约责任等。例如，合同可以采用书面、口头或其他形式订立，内容包括标的、数量、质量等条款。还涉及要约、承诺、定金、违约金等具体规则。

这些规定旨在明确物的归属、保护权利人的合法权益、维护社会经济秩序，并促进社会主义现代化建设。
</translated>
